In [1]:
from sklearn import datasets,svm

In [2]:
iris=datasets.load_iris()

In [3]:
import pandas as pd
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower']=iris.target
df['flower']=df['flower'].apply(lambda x:iris.target_names[x])

In [4]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [5]:
df['flower'].value_counts()

virginica     50
setosa        50
versicolor    50
Name: flower, dtype: int64

# Approach1: Use train test split and manually tune parameters by trial and error

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=0.3)

In [7]:
model=svm.SVC(kernel='rbf',C=30,gamma='auto')

model.fit(x_train,y_train)

model.score(x_test,y_test)

0.9777777777777777

# Approach 2:Use k fold cross validation

In [8]:
from sklearn.model_selection import cross_val_score

In [9]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [10]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

Above approach is very manual ,we can use for loop to automate this

In [11]:
import numpy as np

In [18]:
kernels=['rbf','linear']
C=[1,10,20]
avg_score={}
for kval in kernels:
    for cval in C:
        cv_scores=cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data,iris.target,cv=5)
        avg_score[kval+' '+str(cval)]=np.average(cv_scores)
avg_score

{'rbf 1': 0.9800000000000001,
 'rbf 10': 0.9800000000000001,
 'rbf 20': 0.9666666666666668,
 'linear 1': 0.9800000000000001,
 'linear 10': 0.9733333333333334,
 'linear 20': 0.9666666666666666}

# Use gridsearch CV

In [21]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(svm.SVC(gamma='auto'),{'C':[1,10,20],'kernel':['rbf','linear']},cv=5,return_train_score=False)

In [22]:
clf.fit(iris.data,iris.target)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear']})

In [24]:
df=pd.DataFrame(clf.cv_results_)

In [25]:
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000504,0.000089,0.000223,0.000040,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000456,0.000121,0.000172,0.000009,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000419,0.000050,0.000209,0.000037,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000421,0.000062,0.000169,0.000008,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000439,0.000045,0.000193,0.000014,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000345,0.000021,0.000157,0.000012,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [27]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [28]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [29]:
clf.best_score_

0.9800000000000001